# Scala & its Ecosystem 

### Part 1: **Polynote + request + uJson**

### **Dependencies**

***Python:**<br>*


* matplotlib (plotting) - [https://matplotlib.org/](https://matplotlib.org/)
* pyFunctional (functional programming with collections in a data pipeline style) - [https://www.pyfunctional.org/](https://www.pyfunctional.org/)


**Scala:**


* 
  request (request HTTP) - [https://github.com/lihaoyi/requests-scala](https://github.com/lihaoyi/requests-scala)
  
  
  
* 
  uJson (decode json response) - [https://www.lihaoyi.com/post/uJsonfastflexibleandintuitiveJSONforScala.html](https://www.lihaoyi.com/post/uJsonfastflexibleandintuitiveJSONforScala.html)
  
  
  


<br>
The goal of this notebook is to request weather data with Scala (request + uJson) and plot them with Python (matplotlib + pyfunctional).<br>We will experiment the interaction from Python to Scala and vice versa using JEP through Polynote.



# Scala



### Imports




In [3]:
import java.util
import java.lang.{ Double => JDouble }
import java.lang.{ Integer => JInt }
import java.lang.{ Long => JLong }
import java.util.Locale
import java.time.Instant
import java.time.temporal.ChronoUnit
import ujson.Value
import java.time.format.{ DateTimeFormatter, FormatStyle }
import polynote.runtime.python.PythonObject
import scala.collection.JavaConverters._  

> Customisable parameters




In [5]:
val (lat, lon) = (51.925120, 4.478390) // Lunatech Rotterdam: Hofplein 20, 3032 AC Rotterdam

In [7]:
val apiKeyOpenWeather = // API KEY

### Domain Definition




We define the basics to request the OpenWeatherMap API.<br>We need:


* the uri
* the header that define the format of the response, `json` here
* your private key




In [10]:
object OpenWeatherMap {
  private val days = List.range(0, 6).map(index => Instant.now().minus(index, ChronoUnit.DAYS)) // UTC
  private val uriS = days.map(day => s"https://api.openweathermap.org/data/2.5/onecall/timemachine?lat=$lat&lon=$lon&units=metric&dt=${day.getEpochSecond}&appid=$apiKeyOpenWeather")
  private val headers = Map("Accept" -> "application/json")
  
  private def search(uri: String): Value = {
    val response = requests.get.stream(uri, headers = headers)
    ujson.read(response)("current")
  }

  def search(): List[Value] = {
      uriS.map(uri => search(uri))
  }
}

The last thing we need is the DTO to unpack the response from the api and from Python.




In [12]:
final case class WeatherData(ts: Instant, temperature: Double, feelLikeTemp: Double, humidity: Int, windSpeed: Double) {
    override def toString(): String =    
        s"""| Timestamp = $ts (UTC)
            | Temperature = $temperature Celsius 
            | Feel Like temperature = $feelLikeTemp Celsius 
            | Humidity = $humidity % 
            | Wind speed = $windSpeed m/s""".stripMargin
}

object WeatherData {
    // From JSON
    def apply(jsonValue: ujson.Value): WeatherData = {
        val temperature = jsonValue("temp").num
        val feelLikeTemp = jsonValue("feels_like").num
        val humidity = jsonValue("humidity").num.toInt
        val windSpeed = jsonValue("wind_speed").num
        val ts = Instant.ofEpochSecond(jsonValue("dt").num.toLong)
        WeatherData(ts, temperature, feelLikeTemp, humidity, windSpeed)
    }

    // From Python types
    def apply(pythonValue: PythonObject): WeatherData = {
        WeatherData(Instant.ofEpochSecond(pythonValue.ts.timestamp().as[JDouble].toLong), 
            pythonValue.temperature.as[JDouble], 
            pythonValue.feelLikeTemp.as[JDouble], 
            pythonValue.humidity.as[JInt], 
            pythonValue.windSpeed.as[JDouble])
    }
}

#### We request the weather of the current and last 5 days.


We convert the Scala List to a Java List because of compatibility issues with JEP during the transformation from Scala to Python.<br>


https://github.com/ninia/jep/wiki/How-Jep-Works -> Nice summarise on how JEP works<br>




In [14]:
val responses = OpenWeatherMap.search()
val datas: java.util.List[WeatherData] = responses.map(WeatherData(_)).asJava

In [15]:
println(datas.get(0))

### We now have our WeatherData instances and we will use them in Python 👇




# Python




### Imports



In [19]:
from dataclasses import dataclass
from functional import seq
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from matplotlib.legend_handler import HandlerLine2D

### We can access all the fields of a Scala case class with `name()`.



In [21]:
datas.get(0).ts()

## Domain Definition



In [23]:
def formatScalaInstant(ts):
    return datetime.fromtimestamp(ts.getEpochSecond())

# Frozen allow to have immutable fields
# Pretty useful to have dataclass in Python now
@dataclass(frozen=True)
class WeatherDataPython:
    ts: 'datetime'
    temperature: float
    feelLikeTemp: float
    humidity: int
    windSpeed: float

### We are converting Scala `case class` to Python `data class`.



In [25]:
weatherDatas = seq(datas).map(lambda w: WeatherDataPython(formatScalaInstant(w.ts()), w.temperature(), w.feelLikeTemp(), w.humidity(), w.windSpeed()))

In [26]:
for weather in weatherDatas:
    print(weather)

## Plotting



Plot the temperature again the feel like temperature.

In [29]:
def plotExample(ts, temperature, feelLikeTemperature):
    plt.close()
    fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 10))
    l1, = ax1.plot(ts, temperature, label="Temperature °C", linestyle='--', marker = 'o', color='red')
    l2, = ax2.plot(ts, feelLikeTemperature, label="Feel Like Temperature °C", linestyle='-', marker = 'o', color='blue')
    plt.legend(handles=[l1, l2], title="Legend", fancybox=True, shadow=True, fontsize='large', loc='best', bbox_to_anchor=(0.51, 0.75, 0.5, 0.5))
    plt.gcf().autofmt_xdate()
    plt.show()

Plot everything together just as an example for this notebook.

In [31]:
def plotAll(ts, humidity, temperature, feelLikeTemperature, windSpeed):
    plt.close()
    plt.figure(figsize=(10,10))
    l1, = plt.plot(ts, temperature, label="Temperature °C")
    l2, = plt.plot(ts, humidity, label="Humidity %")
    l3, = plt.plot(ts, feelLikeTemperature, label="Feel Like Temperature °C")
    l4, = plt.plot(ts, windSpeed, label="Wind Speed m/s")
    plt.legend(handles=[l1, l2, l3, l4], title="Legend", fancybox=True, shadow=True, fontsize='large', loc='best', bbox_to_anchor=(0.5, 0.5, 0.5, 0.5))
    plt.gcf().autofmt_xdate()
    plt.show()

In [32]:
# TODO: Not very optimized but it is not important for that example
listTimestamp = weatherDatas.fold_left([], lambda current, next: np.append(current, next.ts))
listHumidity = weatherDatas.fold_left([], lambda current, next: np.append(current, next.humidity))
listTemperature = weatherDatas.fold_left([], lambda current, next: np.append(current, next.temperature))
listFeelLikeTemperature = weatherDatas.fold_left([], lambda current, next: np.append(current, next.feelLikeTemp))
listWindSpeed = weatherDatas.fold_left([], lambda current, next: np.append(current, next.windSpeed))

In [33]:
plotExample(listTimestamp.to_list(), listTemperature.to_list(), listFeelLikeTemperature.to_list())

In [34]:
plotAll(listTimestamp.to_list(), listHumidity.to_list(), listTemperature.to_list(), listFeelLikeTemperature.to_list(), listWindSpeed.to_list())

## Go back to Scala?



We have a list of WeatherDataPython and we want a List[WeatherData]

In [37]:
weatherT = weatherDatas.to_list()
print(weatherT)

In [38]:
val listWeathers = weatherT.asScalaList.map(WeatherData(_))
println(listWeathers.head)